In [1]:
from importlib.metadata import version
print("torch version:", version("torch"))

torch version: 2.6.0+cu124


In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


# Implementing a GPT model from scrath to generate text

## 4.1 - Coding an LLM architecture

  - GPT (generative pretrained transformer), are large deep neural network architectures designed to generate new text one word at a time.

In [3]:
#   Specify the configuration of the small GPT-2 model via the following
# Python dictionary, which we will use in the code examples later:
GPT_CONFIG_124M = {
    "vocab_size": 50257,    # Vocabulary size
    "context_length": 1024, # Context length
    "emb_dim": 768,         # Embedding dimension
    "n_heads": 12,          # Number of attention heads
    "n_layers": 12,         # Number of layers
    "drop_rate": 0.1,       # Dropout rate
    "qkv_bias": False       # Query-Key-Value bias
}

  - The GPT_CONFIG_124M dictionary, we use concise variable names for clarity and to prevent long lines of code
    - vocab_size : refers to a vocabulary of 50257 words, as used by the BPE tokenizer
    - context_length : denotres the maximum number of input tokens the model can handle via the positional embeddings
    - emb_dim : represents the embedding size, transforming each token into a 768-dimensional vector.
    - n_heads : indicates the count of attention heads in the multi-head attention mechanism
    - n_layer : specifies the number of transformer blocks in the model, which we will cover in the upcoming discussion.
    - drop_rate : indicates the intensity of the dropout mechanism (0.1 implies a 10% random drop_out of hidden units) to prevent overfitting
    - qkv_bias determines whether to include a bias vector in the Linear layers of the multi-head attention for query, key, and value computations.  We will initially disable this, following the norms of modern LLMs

In [ ]:
# A placeholder GPT model architecture class
import torch
import torch.nn as nn